# Boekenclassificatie

Aan jou de taak om een neuraal netwerk te bouwen dat zo goed mogelijk kan voorspellen in welk genre een sample van een boek zich bevind. Laat met een cross validation matrix zien hoe goed je model zich gedraagt per genre.

## 1. Business Understanding

- Data van [Project Gutenberg](https://www.gutenberg.org/)
- [Bibliotheek van Congress-classificatie](https://www.loc.gov/catdir/cpso/lcc.html)

## 1.1. Aangeleverde data


## 1.2. Genres
De boeken zijn ingedeeld onder de volgende zeven genres:
|Genre                                              |Engelse Vertaling                              |Aantal boeken
|---------------------------------------------------|-----------------------------------------------|-------------
|Amerikaanse Literatuur                             |American Literature                            |4480
|Engelse Literatuur                                 |English Literature                             |4214
|Fictie en jeugdliteratuur                          |Fiction and juvenile belles lettres            |2624
|Geschiedenis van Europa, Azië, Afrika en Oceanië   |History of Europa, Asia, Africa and Oceania    |2071
|Filosofie, Psychologie en Religie                  |Philosophy, Psychology and Religion            |1550
|Taal en Literatuur                                 |Language and Literature                        |1264
|Tijdschriften                                      |Periodicals                                    |1132

## 2. Data Understanding

### 2.1. Importeren van benodigde pakketten

In [45]:
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer

import itertools
import numpy as np
import pandas as pd
import tensorflow as tf

### 2.2. Inspecteren van `training.csv`

In [3]:
training_df = pd.read_csv("Data/huiswerk/training.csv.gz")

Om een eerste blik te werpen op de traindata, gebruik ik de volgende methodes. Ik zie dat er zo’n 120 duizend regels zijn, elk met:
- Een `usage`-veld, waarschijnlijk voor `training.csv` alleen maar `train`, voor `testing.csv` alleen maar `testing`, etc.
- Een `main genre`-veld, dat waarschijnlijk overeenkomt met de genres uit de Business Understanding
- Een `samplenumber`-veld, elk boek is gesplitst per 600 woorden, deze geeft aan wat eindpunt is van de genomen sample is
- Een `txt`-veld, met maximaal 600 woorden

In [ ]:
training_df.info()
training_df.head()

In verband met performance, pak ik eerst 20 duizend items, in plaats van de totale hoeveelheid.

In [5]:
training_df = training_df.head(20000)

#### 2.2.1. `train`-veld
Het klopt inderdaad dat dit bestand alleen `train` als `usage` bevat:

In [ ]:
unique_usages = [[x] for x in training_df['usage'].unique()]
pd.DataFrame(unique_usages, columns=['Usage'])

#### 2.2.2. `main genre`-veld
Om te testen of de genres overeenkomen met de zeven uit de Business Understanding, pak ik de unieke genres van de dataframe, en zie dat ze inderdaad overeenkomen met de juiste uit de Business Understanding:

In [ ]:
unieke_genres = training_df['main genre'].unique()
pd.DataFrame({'Genre': unieke_genres})

#### 2.2.3. `txt`-veld
Het `txt`-veld bevat een deel van de tekst van een boek. Elk boek is namelijk gesplitst per 600 woorden, dus het veld bevat een deel van het boek. Zoals je je hieronder kunt zien, slaan de vijf zinnen los nergens op (althans het einde ervan), maar als je dan verderleest op de volgende regel, zul je zien dat het wél klopt:

In [ ]:
for x in training_df['txt'].head():
    print('TXT: ' + x)

Om een beeld te krijgen van het aantal tekens in het veld, pak ik de standaardafwijking, het minimum en maximum. Hier zie ik dat er een verschoven verdeling zit in het aantal tekens per `txt` veld:

In [ ]:
txt_lengtes = training_df['txt'].apply(lambda txt: len(txt))
print(f"Standaardafwijking van het `txt`-veld is: {txt_lengtes.std()} tekens")
print(f"Gemiddelde van het `txt`-veld is: {txt_lengtes.mean()} tekens")
print(f"Met een minimum van {txt_lengtes.min()} teken(s)")
print(f"En een maximum van {txt_lengtes.max()} tekens")

Omdat het aantal tekens wat verschoven is, vind ik het interessant om te kijken naar het aantal woorden per `txt`, waar ik zie dat het aantal woorden inderdaad rond de 600 zit, maar er toch een aantal regels zijn die niet zoveel bevatten:

In [ ]:
training_df['woordaantal'] = training_df['txt'].apply(lambda txt: len(txt.split()))
txt_lengtes = training_df['woordaantal']
print(f"Standaardafwijking van het `txt`-veld is: {txt_lengtes.std()} woorden")
print(f"Gemiddelde van het `txt`-veld is: {txt_lengtes.mean()} tekens")
print(f"Met een minimum van {txt_lengtes.min()} woord(en)")
print(f"En een maximum van {txt_lengtes.max()} woorden")

Hmm, het lijkt erop dat er een aantal regels zijn met weinig woorden, maar hoeveel zijn dat er? Hieronder zien we dat het grootste deel wel 600 woorden lang is, en maar $0.07 \%$ daaronder zit.

In [ ]:
ranges = []
percentages = []
counts = []
for x in txt_lengtes.groupby(txt_lengtes // 100):
    percentage = (len(x[1]) / len(txt_lengtes) * 100)
    ranges.append(f'{x[0]}00-{x[0]+1}00')
    percentages.append(f"{percentage:2.2f}%")
    counts.append(len(x[1]))

pd.DataFrame({'Reeksen': ranges, 'Percentages': percentages, 'Aantallen': counts})

Om de tokenizer te kunnen laten werken in het model, is het belangrijk om het aantal verschillende woorden ongeveer te weten:

In [ ]:
tmp = set(itertools.chain.from_iterable(training_df['txt'].apply(lambda txt: txt.split())))
len(tmp)

### 2.3. Inladen van `testing.csv` en `validation.csv`

In [ ]:
testing_df = pd.read_csv("Data/huiswerk/testing.csv.gz")
testing_df = testing_df.head(20000)
testing_df.info()

In [ ]:
validation_df = pd.read_csv("Data/huiswerk/validation.csv.gz")
validation_df = validation_df.head(20000)
validation_df.info()

In [21]:
testing_df['woordaantal'] = testing_df['txt'].apply(lambda txt: len(txt.split()))

In [22]:
validation_df['woordaantal'] = validation_df['txt'].apply(lambda txt: len(txt.split()))

## 3. Data Preparation

### 3.1. Filteren van teksten onder de 600 woorden
Zoals we zagen bij de Data Understanding, zijn de meeste teksten precies 600 woorden. Alles wat daaronder zit is een miniscule hoeveelheid, en filter ik daarom gewoon weg.

In [ ]:
AANTAL_WOORDEN_PER_TEKST = 600
training_df = training_df[training_df['woordaantal'] == AANTAL_WOORDEN_PER_TEKST]
training_df.info()

In [23]:
testing_df = testing_df[testing_df['woordaantal'] == AANTAL_WOORDEN_PER_TEKST]
validation_df = validation_df[validation_df['woordaantal'] == AANTAL_WOORDEN_PER_TEKST]

### 3.2. Omzetten labels
Om te kunnen werken met getallen bij de labels, gebruik ik de [`LabelEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) van _scikit-learn_:

In [26]:
le = LabelEncoder()
le.fit(unieke_genres)
LABEL_COUNT = len(unieke_genres)

train_labels = le.transform(training_df['main genre'])
testing_labels = le.transform(testing_df['main genre'])
validation_labels = le.transform(validation_df['main genre'])

### 3.2. Opzetten tokenizer

In [53]:
AANTAL_WOORDEN = 200000
tokenizer = Tokenizer(num_words=AANTAL_WOORDEN)
tokenizer.fit_on_texts(training_df['txt'])

In [ ]:
list(tokenizer.word_index.items())[:5]

In [55]:
train_sequences = tokenizer.texts_to_sequences(training_df['txt'])
testing_sequences = tokenizer.texts_to_sequences(testing_df['txt'])
validation_sequences = tokenizer.texts_to_sequences(validation_df['txt'])

In [ ]:
del sequences
del percentage
del percentages
del ranges
del txt_lengtes
del unieke_genres
del unique_usages
del x
del testing_df
del tmp
del training_df
del validation_df


## 4. Modeling

In [ ]:
DIMS=100
model = models.Sequential([
    layers.Embedding(input_dim=AANTAL_WOORDEN, output_dim=DIMS, input_length=AANTAL_WOORDEN_PER_TEKST),
    layers.Bidirectional(layers.GRU(32, return_sequences=True, dropout=0.2)),
    layers.Bidirectional(layers.GRU(32)),
    layers.Dense(LABEL_COUNT, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device("/cpu:0"):
    history2 = model.fit(train_sequences,
                         train_labels,
                        #  validation_data=(validation_sequences, validation_labels),
                         epochs=10)

In [ ]:
#plot de accuracy en validated accuracy
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
epochs=range(1, len(acc)+1)
plt.plot(epochs, acc, label='accuracy')
plt.plot(epochs, val_acc, label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

## 5. Evaluation

## 6. Deployment